In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv


In [7]:
# read the dataset
df = pd.read_csv('/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv')

In [8]:
#print dataset
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df.shape

(10000, 14)

In [9]:
df.info()
# check the datset if any value is missing or not

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [10]:
#check the duplicated row
df.duplicated().sum()

0

In [11]:
# how many customer leave the bank
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [12]:
# how many catogerys in geographical colums 
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [13]:
# chech the how many male and female
df['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [15]:
# first 3 colums are not use so drop it
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
# convert 2 catagoriacal colums into numarical
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


In [19]:
# scale the value beacuse in dataset some are very big value and some are very small
# if we train the NN then weights are not converge
X = df.drop(columns=['Exited'])
y = df['Exited']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [38]:
X_train.shape

(8000, 11)

In [41]:
# now scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.fit_transform(X_test)

In [42]:
x_train_scaled

array([[-0.23082038, -0.94449979, -0.70174202, ...,  1.71490137,
        -0.57273139,  0.91509065],
       [-0.25150912, -0.94449979, -0.35520275, ..., -0.58312392,
        -0.57273139, -1.09278791],
       [-0.3963303 ,  0.77498705,  0.33787579, ...,  1.71490137,
        -0.57273139, -1.09278791],
       ...,
       [ 0.22433188,  0.58393295,  1.3774936 , ..., -0.58312392,
        -0.57273139, -1.09278791],
       [ 0.13123255,  0.01077067,  1.03095433, ..., -0.58312392,
        -0.57273139, -1.09278791],
       [ 1.1656695 ,  0.29735181,  0.33787579, ...,  1.71490137,
        -0.57273139,  0.91509065]])

## now ready to make model

In [43]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [53]:
# model building
# seqantial model
model = Sequential()

In [55]:
# make nural network of 3 layers: input hidden output
# hidden layer: 3 node with sigmoid activation and 11 input
model.add(Dense(3,activation='sigmoid',input_dim=11))
# output layer: 1 node with sigmoid actication
model.add(Dense(1,activation='sigmoid'))

In [56]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 3)                 36        
                                                                 
 dense_9 (Dense)             (None, 1)                 4         
                                                                 
Total params: 40 (160.00 Byte)
Trainable params: 40 (160.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
# now compile the model
model.compile(loss='binary_crossentropy',optimizer='Adam')

In [59]:
# now fit the model
model.fit(x_train_scaled,y_train,epochs=10)

Epoch 1/10
250/250 [==============================] - 2s 2ms/step - loss: 0.5979
Epoch 2/10
250/250 [==============================] - 1s 2ms/step - loss: 0.5024
Epoch 3/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4664
Epoch 4/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4502
Epoch 5/10
250/250 [==============================] - 1s 3ms/step - loss: 0.4407
Epoch 6/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4348
Epoch 7/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4309
Epoch 8/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4280
Epoch 9/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4259
Epoch 10/10
250/250 [==============================] - 1s 2ms/step - loss: 0.4242


In [60]:
# upper to lower the epoch's loss is decrease

In [61]:
model.layers[0].get_weights()

[array([[ 0.14797416,  0.1760983 ,  0.15253845],
        [-0.5888326 ,  1.5076953 , -1.6670711 ],
        [-0.07030601,  0.08891258,  0.36769333],
        [-0.11922569, -0.05591058, -0.70453703],
        [-0.03425631, -0.43188635, -0.09247443],
        [-0.15362279, -0.35600963, -0.01192115],
        [ 0.9739697 , -0.03462952,  0.50705475],
        [-0.27307025, -0.07090141,  0.33494803],
        [-0.72609675,  0.09118894, -0.15355872],
        [ 0.06144051, -0.14642198, -0.26569632],
        [ 0.11087842, -0.53185487,  0.57339454]], dtype=float32),
 array([ 0.5212621 , -0.36142588,  0.4701291 ], dtype=float32)]

In [62]:
model.layers[1].get_weights()

[array([[-1.8185424],
        [ 1.0781397],
        [-1.0168195]], dtype=float32),
 array([-0.39201644], dtype=float32)]

In [63]:
# Now model is trained and we get the values of each weights and bius

In [68]:
y_log = model.predict(x_test_scaled)

63/63 [==============================] - 0s 1ms/step


In [69]:
y_log
# the output are between 0 to 1 means probability bcz we use sigmoid function

array([[0.10682978],
       [0.1386434 ],
       [0.13764   ],
       ...,
       [0.05494166],
       [0.12648256],
       [0.39887893]], dtype=float32)

In [70]:
# now we can convert it into 0 to 1 using thresold
y_pred = np.where(y_log>0.5,1,0)

In [71]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [73]:
# now time to check the accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.816

In [74]:
# now change the nural network to get high accuracy: use rlue funtion, layers's node incress and also change the epochs

In [77]:
model1 = Sequential()
# make nural network of 3 layers: input hidden output
# hidden layer: 11 node with relu activation and 11 input
model1.add(Dense(11,activation='relu',input_dim=11))
# hidden layer: 11 node with relu activation
model1.add(Dense(11,activation='relu'))
# output layer: 1 node with sigmoid actication
model1.add(Dense(1,activation='sigmoid'))

In [78]:
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 11)                132       
                                                                 
 dense_14 (Dense)            (None, 11)                132       
                                                                 
 dense_15 (Dense)            (None, 1)                 12        
                                                                 
Total params: 276 (1.08 KB)
Trainable params: 276 (1.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [79]:
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [82]:
model1.fit(x_train_scaled,y_train,epochs=100,validation_split=0.2)

Epoch 1/100
200/200 [==============================] - 2s 4ms/step - loss: 0.6118 - accuracy: 0.6775 - val_loss: 0.5078 - val_accuracy: 0.7962
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 0.4712 - accuracy: 0.7967 - val_loss: 0.4529 - val_accuracy: 0.7969
Epoch 3/100
200/200 [==============================] - 1s 3ms/step - loss: 0.4372 - accuracy: 0.8011 - val_loss: 0.4327 - val_accuracy: 0.7962
Epoch 4/100
200/200 [==============================] - 1s 3ms/step - loss: 0.4213 - accuracy: 0.8073 - val_loss: 0.4192 - val_accuracy: 0.8075
Epoch 5/100
200/200 [==============================] - 1s 3ms/step - loss: 0.4096 - accuracy: 0.8164 - val_loss: 0.4080 - val_accuracy: 0.8138
Epoch 6/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3992 - accuracy: 0.8219 - val_loss: 0.3982 - val_accuracy: 0.8206
Epoch 7/100
200/200 [==============================] - 1s 3ms/step - loss: 0.3907 - accuracy: 0.8277 - val_loss: 0.3903 - val_accuracy: 0.8238

In [83]:
y_log1 = model.predict(x_test_scaled)

63/63 [==============================] - 0s 2ms/step


In [84]:
y_pred1 = np.where(y_log1>0.5,1,0)

In [85]:
y_pred1

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [86]:
accuracy_score(y_test,y_pred1)

0.816